# 날짜와 시간 다루기

## 1. 문자열을 날짜로 변환하기

In [1]:
#라이브러리
import numpy as np
import pandas as pd

#문자열
date_strings=  np.array(['03-04-2005 11:35 PM',
                        '23-05-2010 12:01 PM',
                        '04-09-2009 09:09 PM'])

In [3]:
[pd.to_datetime(date, format='%d-%m-%Y %I:%M %p') for date in date_strings]

[Timestamp('2005-04-03 23:35:00'),
 Timestamp('2010-05-23 12:01:00'),
 Timestamp('2009-09-04 21:09:00')]

In [7]:
#errors매개변수를 추가할 경우
[pd.to_datetime(date,format ='%d-%m-%Y %I:%M %p',errors= 'ignore')for date in date_strings]

[Timestamp('2005-04-03 23:35:00'),
 Timestamp('2010-05-23 12:01:00'),
 Timestamp('2009-09-04 21:09:00')]

 + errors = "raise" => 문제가 있을 때 예외를 발생시킨다. 
 + errors = "coerce" => 문제가 발생할 때 에러를 일으키지 않지만 에러가 난 값을 NaT(누락된 값)으로 설정
 + errors = "ignore" => 예외를 발생키기거나 NaT를 반환하지 않고 원본 문자열을 그대로 반환한다.

## 2. 시간대 다루기

In [9]:
#datetime을 만든다.
pd.Timestamp('2017-05-01 06:00:00', tz='Europe/London')

Timestamp('2017-05-01 06:00:00+0100', tz='Europe/London')

In [10]:
#to_localize를 사용해 이전에 만든 datetime에 시간대를 추가 할 수 있다.
date= pd.Timestamp('2017-05-01 06:00:00')

In [11]:
#시간대를 지정한다.
date_in_london = date.tz_localize('Europe/London')

In [12]:
date_in_london

Timestamp('2017-05-01 06:00:00+0100', tz='Europe/London')

In [13]:
#다른시간대로 변경
date_in_london.tz_convert('Africa/Abidjan')

Timestamp('2017-05-01 05:00:00+0000', tz='Africa/Abidjan')

In [14]:
#세개의 날짜를 만든다.
dates = pd.Series(pd.date_range('2/2/2002',periods = 3, freq= 'M'))

In [17]:
dates.dt.tz_localize('Africa/Abidjan')

0   2002-02-28 00:00:00+00:00
1   2002-03-31 00:00:00+00:00
2   2002-04-30 00:00:00+00:00
dtype: datetime64[ns, Africa/Abidjan]

In [18]:
#판다스는 두 종류의 시간대 문자열을 지원한다.
#pytz 라이브러리를 사용하는 것이 좋다.

#라이브러리
from pytz import all_timezones

In [19]:
#두개의 시간대 확인
all_timezones[:2]

['Africa/Abidjan', 'Africa/Accra']

In [20]:
dates.dt.tz_localize('dateutil/Asia/Seoul')

0   2002-02-28 00:00:00+09:00
1   2002-03-31 00:00:00+09:00
2   2002-04-30 00:00:00+09:00
dtype: datetime64[ns, tzfile('ROK')]

In [23]:
#또 다른 방법
import pytz
tz = pytz.timezone('Asia/Seoul')
dates.dt.tz_localize(tz)

0   2002-02-28 00:00:00+09:00
1   2002-03-31 00:00:00+09:00
2   2002-04-30 00:00:00+09:00
dtype: datetime64[ns, Asia/Seoul]

## 3.날짜와 시간 선택하기

In [33]:
dataframe = pd.DataFrame()

In [34]:
dataframe['data']=pd.date_range('01/01/2001',periods = 100000,freq = 'H')

In [38]:
dataframe[(dataframe['data'] >'2002-01-01 01:00:00') & 
        (dataframe['data']<='2002-1-1 04:00:00')]

,data
8763,2002-01-01 03:00:00
8764,2002-01-01 04:00:00


In [29]:
#인덱스를 설정한다.
dataframe = dataframe.set_index(dataframe['data'])

In [30]:
dataframe.loc['2002-1-1 01:00:00':'2002-1-1 04:00:00']

,data
data,
2002-01-01 01:00:00,2002-01-01 01:00:00
2002-01-01 02:00:00,2002-01-01 02:00:00
2002-01-01 03:00:00,2002-01-01 03:00:00
2002-01-01 04:00:00,2002-01-01 04:00:00


 + 복잡한 시계열을 사용할 땐 =>날짜열을 데이터 프레임의 열로 설정
 + 간단한 데이터 랭글링 => 불리언 조건

## 4.날짜 데이터를 여러 특성으로 나누기

In [40]:
dataframe = pd.DataFrame()

In [41]:
#150개의 날짜를 만든다.
dataframe['date']=pd.date_range('1/1/2001', periods = 150, freq='W' )

In [44]:
dataframe['year'] = dataframe['date'].dt.year
dataframe['month'] = dataframe['date'].dt.month
dataframe['day'] = dataframe['date'].dt.day
dataframe['hour']=dataframe['date'].dt.hour
dataframe['minute']= dataframe['date'].dt.minute

In [45]:
dataframe.head()

,date,year,month,day,hour,minute
0,2001-01-07,2001,1,7,0,0
1,2001-01-14,2001,1,14,0,0
2,2001-01-21,2001,1,21,0,0
3,2001-01-28,2001,1,28,0,0
4,2001-02-04,2001,2,4,0,0


## 5.날짜 간의 차이를 계산하기

 + 두 지점 사이의 시간 변화를 기록한 특성이 필요할 떄
 + 예시 : 호텔의 체크인 시간, 체크아웃 시간

In [46]:
dataframe = pd.DataFrame()

In [52]:
dataframe['Arrived'] = [pd.Timestamp('01-01-2017'),pd.Timestamp('01/4/2017')]
dataframe['Left']=[pd.Timestamp('1/1/2017'),pd.Timestamp('01-06-2017')]

#날짜를 표시할때 '/' or '-' 두개 다 사용가능하다.

In [53]:
dataframe['Left'] - dataframe['Arrived']

0   0 days
1   2 days
dtype: timedelta64[ns]

In [54]:
#days출력 삭제
pd.Series(delta.days for delta in (dataframe['Left']-dataframe['Arrived']))

0    0
1    2
dtype: int64

## 6.요일을 인코딩하기

In [55]:
dates = pd.Series(pd.date_range('2/2/2002',periods = 3, freq ='M'))

In [56]:
dates

0   2002-02-28
1   2002-03-31
2   2002-04-30
dtype: datetime64[ns]

In [57]:
dates.dt.day_name()

0    Thursday
1      Sunday
2     Tuesday
dtype: object

In [58]:
#요일을 수치형 값으로 출력
#월요일이 0
dates.dt.weekday

0    3
1    6
2    1
dtype: int64

## 7.시차 특성 만들기

 + 과거 값을 사용해서 현재 값을 예측할 때 사용
 + 예시 : 하루 전 주식 가격을 사용해 오늘 가격을 예측하는 경우

In [59]:
dataframe = pd.DataFrame()

In [60]:
dataframe['dates']=pd.date_range('1/1/2001', periods = 5,freq='D')
dataframe['stock_price'] = [1.1,2.2,3.3,4.4,5.5]

In [61]:
#한 행 전의 값을 가져온다.
dataframe['Previous_stock_price'] = dataframe['stock_price'].shift(1)

In [62]:
dataframe

,dates,stock_price,Previous_stock_price
0,2001-01-01,1.1,NaN
1,2001-01-02,2.2,1.1
2,2001-01-03,3.3,2.2
3,2001-01-04,4.4,3.3
4,2001-01-05,5.5,4.4


## 8.이동 시간 윈도 사용하기

 + 이동 시간 윈도 예시
 + 세 달을 이동 시간 윈도로 정할 경우
    + mean(1월,2월,3월)
    + mean(2월,3월,4월)
    + mean(3월,4월,5월))

In [63]:
#datetime을 만든다.
time_index = pd.date_range('1/1/2001',periods= 5,freq='M')

In [64]:
dataframe = pd.DataFrame(index = time_index)

In [66]:
dataframe['Stock_Price']=[1,2,3,4,5]

In [67]:
dataframe

,Stock_Price
2001-01-31,1
2001-02-28,2
2001-03-31,3
2001-04-30,4
2001-05-31,5


In [71]:
#이동 평균을 계산한다.
dataframe.rolling(window= 2).mean()

,Stock_Price
2001-01-31,NaN
2001-02-28,1.5
2001-03-31,2.5
2001-04-30,3.5
2001-05-31,4.5


## 9.시계열 데이터에서 누락된 값 다루기

In [72]:
#날짜를 만든다.
time_index = pd.date_range('1/1/2010',periods = 5,freq='M')

In [73]:
dataframe = pd.DataFrame(index = time_index)

In [74]:
dataframe['Sales']=[1.0,2.0,np.nan, np.nan,5.0]

In [75]:
dataframe

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,NaN
2010-04-30,NaN
2010-05-31,5.0


In [77]:
#누락된 값을 보간한다.
dataframe.interpolate()

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,3.0
2010-04-30,4.0
2010-05-31,5.0


 + 보간 : 누락된 값의 양쪽 경계를 잇는 직선이나 곡선을 사용해 적절한 값을 예측함으로써 비어 있는 간격을 채우는 방법

In [78]:
#누락된 값을 앞쪽으로 채우기(Forward-fill)
dataframe.ffill()

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,2.0
2010-04-30,2.0
2010-05-31,5.0


In [79]:
#누락된 값을 뒤쪽으로 채우기(Back -fill)
dataframe.bfill()

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,5.0
2010-04-30,5.0
2010-05-31,5.0


  + 두 포인트 사이의 직선이 비선형일때 interpolate의 method 매개변수를 사용해 다른 보간 방법을 선택할 수 있다.
 

In [80]:
dataframe.interpolate(method='quadratic')

,Sales
2010-01-31,1.000000
2010-02-28,2.000000
2010-03-31,3.059808
2010-04-30,4.038069
2010-05-31,5.000000


In [81]:
# 누락된 값의 간격이 클 떄
# limit 매개변수로 보간 값의 개수를 제한하고,
# limit_direction 매개변수로 방향을 지정한다.
dataframe.interpolate(limit = 1, limit_direction='forward')

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,3.0
2010-04-30,NaN
2010-05-31,5.0
